# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [8]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../").resolve()))
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_cyber_question import OrganizationCyberModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd


In [9]:
sqlstorage = SQLiteStorageProvider("organization_cyber.db")

workflow = Workflow(
    SonarQueryHandler(OrganizationCyberModel),
    sqlstorage,
    workers = 4,
    )

data = pd.read_excel("organization_names_justice.xlsx")
organizations = data["organization_name"].tolist()
countries = data["country"].tolist()

question_set = get_question_set(
    organizations=organizations,
    countries=countries,
)

question_set.max_questions = 0  # No limit

Loaded 3 stored question hashes from organization_cyber.db


In [10]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 193 questions with 4 workers
Processing question 1/193: 
When did the {organization_country} become responsible for cybersecurity?

A ministry handles cyber if it: Is explicitly mentioned in a national strategy/law/report as being responsible for cybersecurity policy, implementation, or technical coordination; Hosts a national CERT/CSIRT/CIRT; Leads or is a member of a cybersecurity committee, council, or working group; Oversees information security standards, network protection, or the like; Attends or participates in events, workshops, or press releases; Works with other countries or organizations on joint initiatives; and other possible forms.
 - {'organization_country': 'Ministry of Justice in SENEGAL'}
Processing question 2/193: 
When did the {organization_country} become responsible for cybersecurity?

A ministry handles cyber if it: Is explicitly mentioned in a national strategy/law/report as being responsible for

[]

In [11]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,organization_country,organization,responsibility_level,explanation,confidence,enriched_citations,pretty_citations
0,\nWhen did the Ministry of Justice and Constit...,None,Ministry of Justice and Constitutional Affairs...,Ministry of Justice and Constitutional Affairs...,ResponsibilityLevel.NONE,The search results do not mention the Ministry...,ConfidenceLevel.HIGH,[{'url': 'https://privacyinternational.org/sit...,[1] [PDF] The Right to Privacy in the United R...
1,\nWhen did the Ministry of Justice of the Isla...,None,Ministry of Justice of the Islamic Emirate of ...,Ministry of Justice of the Islamic Emirate of ...,ResponsibilityLevel.LOW,The Ministry of Justice (MoJ) in Afghanistan h...,ConfidenceLevel.HIGH,[{'url': 'https://mcit.gov.af/sites/default/fi...,[1] [PDF] National Cyber Security Strategy of ...
2,\nWhen did the Ministry of Justice / Ministria...,None,Ministry of Justice / Ministria e Drejtësisë i...,Ministria e Drejtësisë (Ministry of Justice),ResponsibilityLevel.NONE,Based on the latest available legislative docu...,ConfidenceLevel.HIGH,[{'url': 'https://aksk.gov.al/wp-content/uploa...,[1] [PDF] REPUBLIC OF ALBANIA PARLIAMENT LAW N...
3,\nWhen did the Ministry of Justice in EGYPT be...,None,Ministry of Justice in EGYPT,Ministry of Justice in Egypt,ResponsibilityLevel.NONE,None,ConfidenceLevel.LOW,[{'url': 'https://www.escc.gov.eg/strategy.pdf...,[1] [PDF] National Cybersecurity Strategy1 201...
4,\nWhen did the Ministry of Justice (Kuwait) in...,None,Ministry of Justice (Kuwait) in KUWAIT,Ministry of Justice (Kuwait),ResponsibilityLevel.NONE,The official National Cyber Security Strategy ...,ConfidenceLevel.HIGH,[{'url': 'https://citra.gov.kw/sites/en/LegalR...,[1] [PDF] National Cyber Security Strategy of ...
...,...,...,...,...,...,...,...,...,...
188,\nWhen did the Attorney General's Chambers and...,None,Attorney General's Chambers and Ministry of Ju...,Attorney General's Chambers and Ministry of Ju...,ResponsibilityLevel.NONE,There is no evidence in the available sources ...,ConfidenceLevel.HIGH,[{'url': 'https://www.govt.lc/news/minister-fl...,[1] Minister Fletcher to address cyber securit...
189,\nWhen did the Ministry of Law & Justice (Gove...,None,Ministry of Law & Justice (Government of India...,"Ministry of Law & Justice, Government of India",ResponsibilityLevel.LOW,The Ministry of Law & Justice in India primari...,ConfidenceLevel.HIGH,[{'url': 'https://www.webasha.com/blog/when-di...,[1] When Did Cyber Security Start in India? A ...
190,"\nWhen did the Ministry of Justice, Human Righ...",None,"Ministry of Justice, Human Rights and Institut...","Ministry of Justice, Human Rights and Institut...",ResponsibilityLevel.NONE,There is no evidence in the provided search re...,ConfidenceLevel.HIGH,[{'url': 'https://generisonline.com/an-overvie...,[1] An Overview of Cybersecurity Regulations i...
191,\nWhen did the Ministry of Justice in SWEDEN b...,None,Ministry of Justice in SWEDEN,"Ministry of Justice, Sweden",ResponsibilityLevel.HIGH,The Ministry of Justice in Sweden plays a sign...,ConfidenceLevel.HIGH,[{'url': 'https://www.computerweekly.com/news/...,[1] Sweden to establish national cyber securit...


In [12]:
wrangled = flattened.copy()


In [13]:
wrangled.enriched_citations

0      [{'url': 'https://privacyinternational.org/sit...
1      [{'url': 'https://mcit.gov.af/sites/default/fi...
2      [{'url': 'https://aksk.gov.al/wp-content/uploa...
3      [{'url': 'https://www.escc.gov.eg/strategy.pdf...
4      [{'url': 'https://citra.gov.kw/sites/en/LegalR...
                             ...                        
188    [{'url': 'https://www.govt.lc/news/minister-fl...
189    [{'url': 'https://www.webasha.com/blog/when-di...
190    [{'url': 'https://generisonline.com/an-overvie...
191    [{'url': 'https://www.computerweekly.com/news/...
192    [{'url': 'https://www.moj.gov.sa/English/Minis...
Name: enriched_citations, Length: 193, dtype: object

In [14]:
wrangled.to_excel("organization_cyber_justice.xlsx", index=False)